# Create a Q&A Chatbot with LangChain Project

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.3 MB/s eta 0:00:00

### Set the API Keys as an Environment Variable

In [3]:
%load_ext dotenv
%dotenv

### Import the Libraries

In [4]:
from langchain_community.document_loaders.pdf import PyPDFLoader

from langchain_text_splitters import (MarkdownHeaderTextSplitter,
                                      TokenTextSplitter)

from langchain_core.output_parsers.string import StrOutputParser
from langchain_core.messages import SystemMessage
from langchain_core.prompts import (PromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)

from langchain_core.runnables import (RunnablePassthrough,
                                      RunnableLambda,
                                      chain)

from langchain_groq import ChatGroq
from groq import Groq
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


from langchain_chroma.vectorstores import Chroma

from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

import torch

import time

import os

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Load the Course Transcript

In [6]:
loader_pdf = PyPDFLoader("Introduction_to_Tableau.pdf")

In [7]:
docs_list = loader_pdf.load()
docs_list

[Document(metadata={'source': 'Introduction_to_Tableau.pdf', 'page': 0}, page_content="# Introduction to Tableau \n## Welcome to Tableau \nHi, everyone. \nI'm Ned and I'll be your instructor for this \ncourse. \nTableau is an invaluable tool. \nOne needs to learn on their journey to become a \nsuccessful business intelligence analyst or \ndata scientist. \nThe art of these professions is storytelling \nusing data to tell stories and convince top \nmanagement of the right course of action. \nBy completing this part of the program, you \nwill know how to create charts and dashboards \nin tableaux. \nThis is an essential step on your way to a data \nscientist role. \n \n## Why use Tableau: Make your data make an impact \nTableau has grown to become one of the most \npopular business intelligence tools in the \nentire world. \nIt is A B I software that allows non technical \nusers to visualize their data and work with it \nalmost immediately lowering, \nknow how barriers dramatically in th

In [8]:
print(len(docs_list))
for doc in docs_list:
    print(doc.page_content)

49
# Introduction to Tableau 
## Welcome to Tableau 
Hi, everyone. 
I'm Ned and I'll be your instructor for this 
course. 
Tableau is an invaluable tool. 
One needs to learn on their journey to become a 
successful business intelligence analyst or 
data scientist. 
The art of these professions is storytelling 
using data to tell stories and convince top 
management of the right course of action. 
By completing this part of the program, you 
will know how to create charts and dashboards 
in tableaux. 
This is an essential step on your way to a data 
scientist role. 
 
## Why use Tableau: Make your data make an impact 
Tableau has grown to become one of the most 
popular business intelligence tools in the 
entire world. 
It is A B I software that allows non technical 
users to visualize their data and work with it 
almost immediately lowering, 
know how barriers dramatically in the past. 
Business analysts needed the help of it 
personnel who could assist them in gathering 
raw data and 

In [9]:
string_list_concat = "".join([i.page_content for i in docs_list])
string_list_concat

"# Introduction to Tableau \n## Welcome to Tableau \nHi, everyone. \nI'm Ned and I'll be your instructor for this \ncourse. \nTableau is an invaluable tool. \nOne needs to learn on their journey to become a \nsuccessful business intelligence analyst or \ndata scientist. \nThe art of these professions is storytelling \nusing data to tell stories and convince top \nmanagement of the right course of action. \nBy completing this part of the program, you \nwill know how to create charts and dashboards \nin tableaux. \nThis is an essential step on your way to a data \nscientist role. \n \n## Why use Tableau: Make your data make an impact \nTableau has grown to become one of the most \npopular business intelligence tools in the \nentire world. \nIt is A B I software that allows non technical \nusers to visualize their data and work with it \nalmost immediately lowering, \nknow how barriers dramatically in the past. \nBusiness analysts needed the help of it \npersonnel who could assist them in

### Split the Course Transcript with MarkdownHeaderTextSplitter

In [10]:
md_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[
        ("#", "Section Title"),
        ("##", "Course Title"),
    ]
)

In [11]:
docs_list_md_split = md_splitter.split_text(string_list_concat)

In [12]:
print(docs_list_md_split)

[Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Welcome to Tableau'}, page_content="Hi, everyone.\nI'm Ned and I'll be your instructor for this\ncourse.\nTableau is an invaluable tool.\nOne needs to learn on their journey to become a\nsuccessful business intelligence analyst or\ndata scientist.\nThe art of these professions is storytelling\nusing data to tell stories and convince top\nmanagement of the right course of action.\nBy completing this part of the program, you\nwill know how to create charts and dashboards\nin tableaux.\nThis is an essential step on your way to a data\nscientist role."), Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Why use Tableau: Make your data make an impact'}, page_content="Tableau has grown to become one of the most\npopular business intelligence tools in the\nentire world.\nIt is A B I software that allows non technical\nusers to visualize their data and work with it\nalmost immediately

In [13]:
print(len(docs_list_md_split),end='\n\n') # >> equal the number of pages
for doc in docs_list_md_split: ## split every page with the markdown head
  print(doc.page_content)
  print('\n\n')

25

Hi, everyone.
I'm Ned and I'll be your instructor for this
course.
Tableau is an invaluable tool.
One needs to learn on their journey to become a
successful business intelligence analyst or
data scientist.
The art of these professions is storytelling
using data to tell stories and convince top
management of the right course of action.
By completing this part of the program, you
will know how to create charts and dashboards
in tableaux.
This is an essential step on your way to a data
scientist role.



Tableau has grown to become one of the most
popular business intelligence tools in the
entire world.
It is A B I software that allows non technical
users to visualize their data and work with it
almost immediately lowering,
know how barriers dramatically in the past.
Business analysts needed the help of it
personnel who could assist them in gathering
raw data and preprocessing it.
Only then could business analysts start working
on the visualization of such data?
The advent of Tableau 

### Create a Chain to Correct the Course Transcript

In [14]:
string_list_split = [doc.page_content  for doc in docs_list_md_split]
print(string_list_concat)

# Introduction to Tableau 
## Welcome to Tableau 
Hi, everyone. 
I'm Ned and I'll be your instructor for this 
course. 
Tableau is an invaluable tool. 
One needs to learn on their journey to become a 
successful business intelligence analyst or 
data scientist. 
The art of these professions is storytelling 
using data to tell stories and convince top 
management of the right course of action. 
By completing this part of the program, you 
will know how to create charts and dashboards 
in tableaux. 
This is an essential step on your way to a data 
scientist role. 
 
## Why use Tableau: Make your data make an impact 
Tableau has grown to become one of the most 
popular business intelligence tools in the 
entire world. 
It is A B I software that allows non technical 
users to visualize their data and work with it 
almost immediately lowering, 
know how barriers dramatically in the past. 
Business analysts needed the help of it 
personnel who could assist them in gathering 
raw data and pre

In [15]:
for string in string_list_split:
  print(string)
  print('\n\n')

Hi, everyone.
I'm Ned and I'll be your instructor for this
course.
Tableau is an invaluable tool.
One needs to learn on their journey to become a
successful business intelligence analyst or
data scientist.
The art of these professions is storytelling
using data to tell stories and convince top
management of the right course of action.
By completing this part of the program, you
will know how to create charts and dashboards
in tableaux.
This is an essential step on your way to a data
scientist role.



Tableau has grown to become one of the most
popular business intelligence tools in the
entire world.
It is A B I software that allows non technical
users to visualize their data and work with it
almost immediately lowering,
know how barriers dramatically in the past.
Business analysts needed the help of it
personnel who could assist them in gathering
raw data and preprocessing it.
Only then could business analysts start working
on the visualization of such data?
The advent of Tableau demo

In [16]:
PROMPT_FORMATTING_S = '''Improve the following Tableau lecture transcript by:
- Splitting the text into meaningful paragraphs
- Correcting any misplaced punctuation
- Fixing mistranscribed words (e.g., changing 'tableaux' to 'Tableau')"
'''

PROMPT_TEMPLATE_FORMATTING_H = '''This is the transcript:
{lecture_transcript}
'''

In [17]:
prompt_formatting_s = SystemMessage(content=PROMPT_FORMATTING_S)
prompt_template_formatting_h = HumanMessagePromptTemplate.from_template(
                    template=PROMPT_TEMPLATE_FORMATTING_H)
chat_prompt_template_formatting = ChatPromptTemplate([prompt_formatting_s,prompt_template_formatting_h])

In [18]:
chat_prompt_template_formatting

ChatPromptTemplate(input_variables=['lecture_transcript'], input_types={}, partial_variables={}, messages=[SystemMessage(content='Improve the following Tableau lecture transcript by:\n- Splitting the text into meaningful paragraphs\n- Correcting any misplaced punctuation\n- Fixing mistranscribed words (e.g., changing \'tableaux\' to \'Tableau\')"\n', additional_kwargs={}, response_metadata={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['lecture_transcript'], input_types={}, partial_variables={}, template='This is the transcript:\n{lecture_transcript}\n'), additional_kwargs={})])

In [19]:
repo_id = 'Qwen/Qwen2.5-Coder-32B-Instruct'

HUGGINGFACEHUB_API_TOKEN = os.environ['HUGGINGFACEHUB_API_TOKEN']

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    max_new_tokens=700,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [20]:
str_output_parser = StrOutputParser()

In [21]:
chain_formatting = (chat_prompt_template_formatting
                    | chat
                    | str_output_parser)

In [22]:
len(string_list_split)

25

In [23]:
response = chain_formatting.invoke(string_list_split[0])

print(response)

Hi, everyone. I'm Ned, and I'll be your instructor for this course. Tableau is an invaluable tool that one needs to learn on their journey to becoming a successful business intelligence analyst or data scientist. The art of these professions is storytelling—using data to tell stories and convince top management of the right course of action. By completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data


In [24]:
string_list_formatted = chain_formatting.batch(string_list_split)

print(string_list_formatted)

["Hi, everyone. I'm Ned, and I'll be your instructor for this course. Tableau is an invaluable tool that one needs to learn on their journey to becoming a successful business intelligence analyst or data scientist. The art of these professions is storytelling—using data to tell stories and convince top management of the right course of action. By completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data", 'Tableau has grown to become one of the most popular business intelligence tools in the entire world. It is BI software that allows non-technical users to visualize their data and work with it almost immediately, lowering knowledge barriers dramatically in the past. Business analysts previously required assistance from IT personnel to gather raw data and preprocess it before they could start working on visualizations. The advent of Tableau democratized this process, enabling BI analysts to be inde

In [25]:
print(string_list_formatted[0])

Hi, everyone. I'm Ned, and I'll be your instructor for this course. Tableau is an invaluable tool that one needs to learn on their journey to becoming a successful business intelligence analyst or data scientist. The art of these professions is storytelling—using data to tell stories and convince top management of the right course of action. By completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data


In [26]:
print(string_list_split[0])

Hi, everyone.
I'm Ned and I'll be your instructor for this
course.
Tableau is an invaluable tool.
One needs to learn on their journey to become a
successful business intelligence analyst or
data scientist.
The art of these professions is storytelling
using data to tell stories and convince top
management of the right course of action.
By completing this part of the program, you
will know how to create charts and dashboards
in tableaux.
This is an essential step on your way to a data
scientist role.


In [27]:
print(len(string_list_formatted))
for string in string_list_formatted:
  print(string)
  print('\n\n')

25
Hi, everyone. I'm Ned, and I'll be your instructor for this course. Tableau is an invaluable tool that one needs to learn on their journey to becoming a successful business intelligence analyst or data scientist. The art of these professions is storytelling—using data to tell stories and convince top management of the right course of action. By completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data



Tableau has grown to become one of the most popular business intelligence tools in the entire world. It is BI software that allows non-technical users to visualize their data and work with it almost immediately, lowering knowledge barriers dramatically in the past. Business analysts previously required assistance from IT personnel to gather raw data and preprocess it before they could start working on visualizations. The advent of Tableau democratized this process, enabling BI analysts to be ind

In [28]:
# Override the docs_list_md_split list such that the page_content parameter of each Document objects stores the updated lecture.
for old_doc,new_doc in zip(docs_list_md_split,string_list_formatted):
    old_doc.page_content = new_doc

In [29]:
print(len(docs_list_md_split))

25


In [30]:
for doc in docs_list_md_split:
    print(doc.page_content)
    print('\n\n')

Hi, everyone. I'm Ned, and I'll be your instructor for this course. Tableau is an invaluable tool that one needs to learn on their journey to becoming a successful business intelligence analyst or data scientist. The art of these professions is storytelling—using data to tell stories and convince top management of the right course of action. By completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data



Tableau has grown to become one of the most popular business intelligence tools in the entire world. It is BI software that allows non-technical users to visualize their data and work with it almost immediately, lowering knowledge barriers dramatically in the past. Business analysts previously required assistance from IT personnel to gather raw data and preprocess it before they could start working on visualizations. The advent of Tableau democratized this process, enabling BI analysts to be indepe

### Split the Lectures with TokenTextSplitter

In [31]:
token_splitter = TokenTextSplitter(
    encoding_name="cl100k_base",
    chunk_size=500,
    chunk_overlap=50
)

In [32]:
docs_list_tokens_split = token_splitter.split_documents(docs_list_md_split)

In [33]:
print(docs_list_tokens_split,end='\n\n')
print(len(docs_list_tokens_split))

[Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Welcome to Tableau'}, page_content="Hi, everyone. I'm Ned, and I'll be your instructor for this course. Tableau is an invaluable tool that one needs to learn on their journey to becoming a successful business intelligence analyst or data scientist. The art of these professions is storytelling—using data to tell stories and convince top management of the right course of action. By completing this part of the program, you will know how to create charts and dashboards in Tableau. This is an essential step on your way to a data"), Document(metadata={'Section Title': 'Introduction to Tableau', 'Course Title': 'Why use Tableau: Make your data make an impact'}, page_content='Tableau has grown to become one of the most popular business intelligence tools in the entire world. It is BI software that allows non-technical users to visualize their data and work with it almost immediately, lowering knowledge barriers dr

### Create Embeddings, Vector Store, and Retriever

In [34]:
embedding_model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)


class CustomEmbedding(Embeddings):
    def embed_documents(self, texts):
        """Encode a list of documents into embeddings as lists."""
        return [embedding.tolist() for embedding in embedding_model.encode(texts, show_progress_bar=True)]

    def embed_query(self, text):
        """Encode a single query string into an embedding as a list."""
        return embedding_model.encode(text, show_progress_bar=False).tolist()

embedding = CustomEmbedding()

modules.json:   0%|          | 0.00/378 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/733k [00:00<?, ?B/s]

custom_st.py:   0%|          | 0.00/8.78k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-embeddings-v3:
- custom_st.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model.safetensors:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

In [ ]:
# vectorstore = Chroma.from_documents(
#     documents=docs_list_tokens_split,
#     embedding=embedding,
#     persist_directory="./intro-to-tableau"
# )

In [36]:
vectorstore = Chroma(
    embedding_function=embedding,
    persist_directory="./intro-to-tableau"
)

In [37]:
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k': 2, 'lambda_mult': 0.7}
)

### Create Prompts and Prompt Templates for the Q&A Chatbot Chain

In [38]:
PROMPT_CREATING_QUESTION = '''Lecture: {question_lecture}
Title: {question_title}
Body: {question_body}'''

PROMPT_RETRIEVING_S = '''You will receive a question from a student taking a Tableau course, which includes a title and a body.
The corresponding lecture will also be provided.

Answer the question using only the provided context.

At the end of your response, include the section and lecture names where the context was drawn from, formatted as follows:
Resources:
Section: *Section Title*, Lecture: *Lecture Title*
...
Replace *Section Title* and *Lecture Title* with the appropriate titles.'''

PROMPT_TEMPLATE_RETRIEVING_H = '''This is the question:
{question}

This is the context:
{context}'''

prompt_creating_question = PromptTemplate.from_template(template=PROMPT_CREATING_QUESTION)
prompt_retrieving_s = SystemMessage(PROMPT_RETRIEVING_S)
prompt_template_retrieving_h = HumanMessagePromptTemplate.from_template(template=PROMPT_TEMPLATE_RETRIEVING_H)

chat_prompt_template_retrieving = ChatPromptTemplate([prompt_retrieving_s,prompt_template_retrieving_h])

### Create the First Version of the Q&A Chatbot Chain

In [40]:
chain_retrieving = (prompt_creating_question
                    | RunnableLambda(lambda response: response.text)
                    | {
                        'context': retriever,
                        'question':RunnablePassthrough()
                      }
                    | chat_prompt_template_retrieving
                    | chat
                    | str_output_parser
                    )

In [41]:
result = chain_retrieving.invoke({"question_lecture": "Adding a custom calculation",
                                  "question_title": "Why are we using SUM here? It's unclear to me.",
                                  "question_body": "This question refers to calculating the GM%."})

In [42]:
print(result)

It appears there is no specific context provided for the calculation of GM% (Gross Margin Percentage) in the question. However, based on common practices in Tableau when calculating metrics like GM%, the use of `SUM` is typically to aggregate the total sales and cost of goods sold (COGS) across the dimension of interest, such as products or time periods. Here’s a general explanation:

Gross Margin Percentage is usually calculated as:
\[ \text{GM%} = \left(


### Create a Runnable Function to Format the Context

In [43]:
@chain
def format_context(dictionary):
  new_content=''
  for idx,doc in enumerate(dictionary['context'],1):
      new_content+=f"Document {idx}\n"
      for key,val in doc.metadata.items():
        new_content+= f"{key}: {val}\n"
      new_content+="Content: "+doc.page_content+'\n'
      new_content+='-'*50
  return {'context':new_content,
          'question':dictionary['question']}

In [44]:
chain_retrieving_improved = (prompt_creating_question
                            | RunnableLambda(lambda response: response.text)
                            | {
                                'context': retriever,
                                'question':RunnablePassthrough()
                              }
                            | format_context
                            | chat_prompt_template_retrieving
                            | chat
                            | str_output_parser
                            )

In [45]:
result_improved = chain_retrieving_improved.invoke({"question_lecture": "Adding a custom calculation",
                                                    "question_title": "Why are we using SUM here? It's unclear to me.",
                                                    "question_body": "This question refers to calculating the GM%."})

In [46]:
result_improved

'Based on the provided context, the question seems to focus on the use of the `SUM` function in Tableau for calculating the Gross Margin Percentage (GM%). However, the specific resources of the section and lecture are not provided in your request. Typically, the `SUM` function is used to aggregate values to perform calculations like Gross Margin Percentage, which generally requires summing up sales and costs.\n\nWhen calculating GM%, you typically sum up the total sales and the total cost of goods sold and then compute'

### Stream the Response

In [47]:
result_streamed = chain_retrieving_improved.stream({"question_lecture": "Adding a custom calculation",
                                                    "question_title": "Why are we using SUM here? It's unclear to me.",
                                                    "question_body": "This question refers to calculating the GM%."})

In [48]:
# Create a for-loop to stream the response
for token in result_streamed:
    print(token, end='', flush=True)

Based on the provided context, the question seems to focus on the use of the `SUM` function in Tableau for calculating the Gross Margin Percentage (GM%). However, the specific resources of the section and lecture are not provided in your request. Typically, the `SUM` function is used to aggregate values to perform calculations like Gross Margin Percentage, which generally requires summing up sales and costs.

When calculating GM%, you typically sum up the total sales and the total cost of goods sold and then compute